In [1]:
# Run only if you have errors loading the topquartile module

from pathlib import Path
import sys

root = Path().resolve().parent.parent.parent.parent
sys.path.append(root)

In [2]:
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import (TechnicalCovariateTransform, FundamentalCovariateTransform)
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition

In [3]:
covtrans_config = [((TechnicalCovariateTransform, dict(sma = [20, 30],
                                                       ema = [20, 30],
                                                       momentum_change=True,
                                                       volatility = [20, 30],)))]

labeltrans_config = [(BinaryLabelTransform, dict(label_duration=20,
                                                quantile=0.75))]

partition_config = dict(n_splits=5, gap=20, max_train_size=504, test_size=60, verbose=False)

In [4]:
dataloader = DataLoader(data_id='dec2024', covariate_transform=covtrans_config,
                  label_transform=labeltrans_config, partition_class=PurgedTimeSeriesPartition,
                  partition_kwargs=partition_config)

In [5]:
folds = dataloader.get_cv_folds()

Data not yet processed. Processing now...
Reading data from: /Users/gregruyoga/gmoneycodes/topquartile/topquartile/data/dec2024.csv
Found 342 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


/Users/gregruyoga/gmoneycodes/topquartile/topquartile/modules/datamodule/transforms/label.py:171: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_copy.groupby(level=self.ticker_level_name, group_keys=False)[self.price_column]


Data processing complete.
Partitioning data using PurgedTimeSeriesPartition for 5 splits across 113 tickers.


/Users/gregruyoga/gmoneycodes/topquartile/topquartile/modules/datamodule/dataloader.py:244: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data_grouped_by_ticker = self.data.groupby("ticker")


Fold 0: Train shape (56952, 39), Test shape (6780, 39)
Fold 1: Train shape (56952, 39), Test shape (6780, 39)
Fold 2: Train shape (56952, 39), Test shape (6780, 39)
Fold 3: Train shape (56952, 39), Test shape (6780, 39)
Fold 4: Train shape (56952, 39), Test shape (6780, 39)
Partitioning complete. Generated 5 CV folds.


In [7]:
folds[0][1]

EQY_DVD_YLD_IND  PX_HIGH  PX_LOW  RETURN_COM_EQY  \
TickerIndex DateIndex                                                      
AALI        2023-10-17           5.5694   7250.0  7175.0          6.1205   
            2023-10-18           5.7286   7300.0  7000.0          6.1205   
            2023-10-19           5.6479   7175.0  7025.0          6.1205   
            2023-10-20           5.6678   7125.0  7025.0          6.1205   
            2023-10-23           5.6879   7125.0  7000.0          6.1205   
...                                 ...      ...     ...             ...   
WSKT        2024-01-02           0.3200    212.0   199.0        -52.4353   
            2024-01-03           0.3200    212.0   199.0        -52.4353   
            2024-01-04           0.3200    212.0   199.0        -65.9300   
            2024-01-05           0.3200    212.0   199.0        -65.9300   
            2024-01-07           0.3200    212.0   199.0        -75.3652   

                        PX_TO_BOOK_RATIO  NEWS_SENTIMENT_DAILY_AVG  \
TickerIndex DateIndex                                                
AALI        2023-10-17            0.6375                    0.0000   
            2023-10-18            0.6198                    0.0000   
            2023-10-19            0.6286                    0.0000   
            2023-10-20            0.6264                    0.0000   
            2023-10-23            0.6242                    0.0000   
...                                  ...                       ...   
WSKT        2024-01-02            0.6671                    0.0000   
            2024-01-03            0.6671                    0.0000   
            2024-01-04            0.6671                   -0.1250   
            2024-01-05            0.6671                    0.0000   
            2024-01-07            0.6671                   -0.1068   

                        NEWS_HEAT_PUB_DAVG  TOT_DEBT_TO_TOT_EQY  VWAP_VOLUME  \
TickerIndex DateIndex                                                          
AALI        2023-10-17                 0.0              18.0673     322000.0   
            2023-10-18                 0.0              18.0673    2826800.0   
            2023-10-19                 0.0              18.0673    1188600.0   
            2023-10-20                 0.0              18.0673     327700.0   
            2023-10-23                 0.0              18.0673    1001300.0   
...                                    ...                  ...          ...   
WSKT        2024-01-02                 0.0             556.7789   18553300.0   
            2024-01-03                 0.0             556.7789   18553300.0   
            2024-01-04                 1.0             597.7109   18553300.0   
            2024-01-05                 0.0             597.7109   18553300.0   
            2024-01-07                 0.0             703.5615   18553300.0   

                        EQY_WEIGHTED_AVG_PX  ...       ema_20       ema_30  \
TickerIndex DateIndex                        ...                             
AALI        2023-10-17            7210.3649  ...  7539.134504  7572.506889   
            2023-10-18            7095.6099  ...  7487.788361  7535.570961   
            2023-10-19            7079.7724  ...  7450.856136  7507.469608   
            2023-10-20            7068.7595  ...  7415.060313  7479.568343   
            2023-10-23            7029.6614  ...  7380.292664  7451.854257   
...                                     ...  ...          ...          ...   
WSKT        2024-01-02             202.0616  ...   203.206163   204.104712   
            2024-01-03             202.0616  ...   203.091290   203.968924   
            2024-01-04             202.0616  ...   202.987358   203.841896   
            2024-01-05             202.0616  ...   202.893324   203.723064   
            2024-01-07             202.0616  ...   202.808245   203.611899   

                        volatility_20  volatility_30    roc_126  \
T